In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import time
import numpy as np
import json
import unicodedata

## Get the list of materials

I will only work on French market. 

Final structure of url is as follow: `https://www.doctolib.fr/{specialite}/{ville|departement|region|pays}`

What I need:
* List of specialties
* List of department

In [ ]:
# Get the list of specialities
url='https://www.doctolib.fr/specialities'

headers="user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
headers=dict([i.split(': ') for i in headers.split('\n')])

r.get(url,headers=headers)

In [ ]:
html=r.get(url,headers=headers).content
soup=BeautifulSoup(html)

list_of_specialties=[i["href"].strip('/') for i in soup.select('h1+div a')]
list_of_specialties

In [ ]:
# Get the list of departments
departments=pd.read_csv('../data/departements-francais.csv',sep="\t") 
departments.head()

In [ ]:
index_to_drop=departments.iloc[-2:].index
departments.drop(index_to_drop,axis=0,inplace=True)

In [ ]:
departments.columns=departments.columns.map(str.capitalize)

In [ ]:
list_of_region=set([str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Region']])


In [ ]:
list_of_department=[str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Nom']]



## #1 List of professionals

In [ ]:
base_url='https://www.doctolib.fr'
specialitie=list_of_specialties[0]
department=list_of_department[0]

final_url=f"{base_url}/{specialitie}/{department}"

headers="""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
cache-control: no-cache
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
referer: https://www.doctolib.fr/chirurgien/paris
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
headers=dict([i.split(': ') for i in headers.split('\n')])

# Succeed to reteive a json format for the results of the search
result=r.get(final_url,headers=headers).json()

In [ ]:
result['data']['doctors']

In [ ]:
data=pd.json_normalize(result['data']['doctors'])
data

In [ ]:
#Get the content with all pages
base_url='https://www.doctolib.fr'
specialitie=list_of_specialties[0]
department=list_of_department[1]

df_final=pd.DataFrame()

for i in range(1,50):
    
    final_url=f"{base_url}/{specialitie}/{department}?page={i}"
    
    headers=f"""accept: application/json
    accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
    cache-control: no-cache
    content-type: application/json; charset=utf-8
    cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
    referer: https://www.doctolib.fr/{specialitie}/{department}?page={i}
    user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
    x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
    headers=dict([i.strip().split(': ') for i in headers.split('\n')])
    
    result=r.get(final_url,headers=headers).json()
    if len(result['data']['doctors'])==0:
        data=pd.json_normalize(result['data']['directory_doctors'])
        data=data.rename(columns={'zip_code':'zipcode'})
        data['doctolib_profile']=False
        df_final=df_final.append(data)
        break
    else:
        data=pd.json_normalize(result['data']['doctors'])
        data['doctolib_profile']=True
        df_final=df_final.append(data)
    time.sleep(1)

In [ ]:
df_final

In [ ]:
df_final.columns
# drop columns : 'id', 'is_directory','cloudinary_public_id','exact_match', 'priority_speciality'

In [ ]:
df_final.tail()

In [ ]:
base_url='https://www.doctolib.fr'
specialitie=list_of_specialties[0]
final_url=f"{base_url}/{specialitie}/ile-de-france?page=1"
    
headers=f"""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
cache-control: no-cache
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
referer: https://www.doctolib.fr/
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

result=r.get(final_url,headers=headers).json()

In [ ]:
pd.json_normalize(result)

In [ ]:
# Final scrapping for one specialities but all location
base_url='https://www.doctolib.fr'
specialitie=list_of_specialties[0]

df_final=pd.DataFrame()

for i in range(len(list_of_department)):
    department=list_of_department[i] 
    
    for i in range(1,50):
    
    final_url=f"{base_url}/{specialitie}/{department}?page={i}"
    
    headers=f"""accept: application/json
    accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
    cache-control: no-cache
    content-type: application/json; charset=utf-8
    cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
    referer: https://www.doctolib.fr/{specialitie}/{department}?page={i}
    user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
    x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
    headers=dict([i.strip().split(': ') for i in headers.split('\n')])
    
    result=r.get(final_url,headers=headers).json()
    if len(result['data']['doctors'])==0:
        data=pd.json_normalize(result['data']['directory_doctors'])
        data=data.rename(columns={'zip_code':'zipcode'})
        data['doctolib_profile']=False
        df_final=df_final.append(data)
        break
    else:
        data=pd.json_normalize(result['data']['doctors'])
        data['doctolib_profile']=True
        df_final=df_final.append(data)
    time.sleep(1)

## #2 Get availability

Data needed:
* Link of the profile (to be matched with final DataFrame)
* Next Avaibility